In [26]:
#----------Cell 1----------


from groq import Groq
from dotenv import load_dotenv
import os
import pprint

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [27]:
task_data_path = "../model_output_data/task_json_data"

if os.path.exists(task_data_path):
    print(f"The path '{task_data_path}' exists.")
else:
    print(f"The path '{task_data_path}' does not exist or is malformed.")

The path '../model_output_data/task_json_data' exists.


In [28]:
#----------Cell 2----------


user_input = "Research on python language"
research_context = "Starting fresh research"

In [29]:
#----------Cell 3----------


messages = []
# MAX_MESSAGES = 10   # total, user + assistant



# if len(messages) > MAX_MESSAGES:
#     messages = messages[-MAX_MESSAGES:]




In [30]:
#----------Cell 4----------

system_prompt = f"""
You are creating an initial research plan for the topic: "{user_input}"
Initial Query: "{user_input}"
Research Context: {research_context if research_context else "Starting fresh research"}
Decompose this query into 3–5 actionable research tasks. Return a JSON array with each task having:
• "description": Clear, actionable task (string)
• "priority": 1–10 (integer, higher = more important, default=5)
• "type": "research" (always "research")
Focus on: understanding the topic, gathering information, identifying key aspects, and building foundational knowledge.
Example for "Impacts of Generative AI on Scientific Research":
<answer>
[
{{"description": "Survey major applications of generative AI in scientific discovery", "priority": 8, "type": "research"}},
{{"description": "Identify key papers and institutions leading AI-assisted science research", "priority": 7, "type": "research"}},
{{"description": "Examine methodological advances enabled by generative models in ...", "priority": 6, "type": "research"}},
{{"description": "Assess challenges and ethical considerations of AI-generated scientific results", "priority": 5, "type": "research"}}
]
</answer>
CRITICAL: Wrap JSON in <answer>tags.
Output ONLY valid JSON.


"""

messages.append({"role": "system", "content": system_prompt})

In [31]:
#----------Cell 5----------


messages.append({"role": "user", "content": user_input}) 

task_agent = Groq()
completion = task_agent.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    stream=False
)

reply = completion.choices[0].message.content
print(reply)
messages.append({"role": "assistant", "content": reply})

<answer>
[
{"description": "Investigate the history and evolution of the Python programming language", "priority": 6, "type": "research"},
{"description": "Gather information about Python's syntax, structure, and basic data types", "priority": 5, "type": "research"},
{"description": "Examine the popular use cases and applications of Python, including web development, data science, and automation", "priority": 7, "type": "research"},
{"description": "Identify essential Python libraries and frameworks, such as NumPy, pandas, and Flask", "priority": 9, "type": "research"},
{"description": "Research the role of popular tools and resources in the Python ecosystem, including pip, virtual environments, and Jupyter Notebooks", "priority": 8, "type": "research"}
]
</answer>


In [32]:
print(messages[-1]['content'])

<answer>
[
{"description": "Investigate the history and evolution of the Python programming language", "priority": 6, "type": "research"},
{"description": "Gather information about Python's syntax, structure, and basic data types", "priority": 5, "type": "research"},
{"description": "Examine the popular use cases and applications of Python, including web development, data science, and automation", "priority": 7, "type": "research"},
{"description": "Identify essential Python libraries and frameworks, such as NumPy, pandas, and Flask", "priority": 9, "type": "research"},
{"description": "Research the role of popular tools and resources in the Python ecosystem, including pip, virtual environments, and Jupyter Notebooks", "priority": 8, "type": "research"}
]
</answer>


In [33]:
assistant_output = messages[-1]["content"]




In [34]:
print(assistant_output)

<answer>
[
{"description": "Investigate the history and evolution of the Python programming language", "priority": 6, "type": "research"},
{"description": "Gather information about Python's syntax, structure, and basic data types", "priority": 5, "type": "research"},
{"description": "Examine the popular use cases and applications of Python, including web development, data science, and automation", "priority": 7, "type": "research"},
{"description": "Identify essential Python libraries and frameworks, such as NumPy, pandas, and Flask", "priority": 9, "type": "research"},
{"description": "Research the role of popular tools and resources in the Python ecosystem, including pip, virtual environments, and Jupyter Notebooks", "priority": 8, "type": "research"}
]
</answer>


In [36]:
print(messages[-1]["content"])


<answer>
[
{"description": "Investigate the history and evolution of the Python programming language", "priority": 6, "type": "research"},
{"description": "Gather information about Python's syntax, structure, and basic data types", "priority": 5, "type": "research"},
{"description": "Examine the popular use cases and applications of Python, including web development, data science, and automation", "priority": 7, "type": "research"},
{"description": "Identify essential Python libraries and frameworks, such as NumPy, pandas, and Flask", "priority": 9, "type": "research"},
{"description": "Research the role of popular tools and resources in the Python ecosystem, including pip, virtual environments, and Jupyter Notebooks", "priority": 8, "type": "research"}
]
</answer>


In [40]:


start = assistant_output.find("<answer>") + len("<answer>")
end = assistant_output.find("</answer>")






json_text = assistant_output[start:end].strip()

tasks = json.loads(json_text)

with open(os.path.join(task_data_path, "tasks.json"), "w") as f:
    json.dump(tasks, f, indent=4)

print(tasks)

[{'description': 'Investigate the history and evolution of the Python programming language', 'priority': 6, 'type': 'research'}, {'description': "Gather information about Python's syntax, structure, and basic data types", 'priority': 5, 'type': 'research'}, {'description': 'Examine the popular use cases and applications of Python, including web development, data science, and automation', 'priority': 7, 'type': 'research'}, {'description': 'Identify essential Python libraries and frameworks, such as NumPy, pandas, and Flask', 'priority': 9, 'type': 'research'}, {'description': 'Research the role of popular tools and resources in the Python ecosystem, including pip, virtual environments, and Jupyter Notebooks', 'priority': 8, 'type': 'research'}]
